# e-footprint quickstart

This notebook provides an example scenario that you can use to get familiar with the Python API of efootprint: the daily video consumption of all French households on a big streaming platform.

You will get to describe:

- the infrastructure involved (servers with auto-scaling settings, storage and network)
- the user journey involving 2 steps (Streaming, Upload)
- the usage pattern and the device population that executes it (the laptops of all French households)

## Import the packages

⚠ If this steps fails, remember to run *ipython kernel install --user --name=efootprint-kernel* _inside_ your python virtual environement (initializable with `poetry install`) to be able to select efootprint-kernel as the jupyter kernel.

In [ ]:
# If this hasn’t been done in virtualenv (useful for Google colab notebook)
!pip install efootprint

In [ ]:
import os

from efootprint.abstract_modeling_classes.source_objects import SourceValue, Sources, SourceObject
from efootprint.abstract_modeling_classes.empty_explainable_object import EmptyExplainableObject
from efootprint.core.usage.usage_journey import UsageJourney
from efootprint.core.usage.usage_journey_step import UsageJourneyStep
from efootprint.core.usage.job import Job
from efootprint.core.hardware.server import Server, ServerTypes
from efootprint.core.hardware.storage import Storage
from efootprint.core.usage.usage_pattern import UsagePattern
from efootprint.core.hardware.network import Network
from efootprint.core.hardware.device import Device
from efootprint.core.system import System
from efootprint.constants.countries import Countries
from efootprint.constants.units import u

## Define the infrastructure

### Creating objects manually
An e-footprint object has a name and attributes describing its technical and environmental characteristics:

In [ ]:
storage = Storage(
    "SSD storage",
    carbon_footprint_fabrication_per_storage_capacity=SourceValue(160 * u.kg / u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    power_per_storage_capacity=SourceValue(1.3 * u.W / u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    lifespan=SourceValue(6 * u.years, Sources.HYPOTHESIS),
    idle_power=SourceValue(0 * u.W, Sources.HYPOTHESIS),
    storage_capacity=SourceValue(1 * u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    data_replication_factor=SourceValue(3 * u.dimensionless, Sources.HYPOTHESIS),
    data_storage_duration = SourceValue(2 * u.year, Sources.HYPOTHESIS),
    base_storage_need = SourceValue(100 * u.TB, Sources.HYPOTHESIS),
    fixed_nb_of_instances = EmptyExplainableObject()
    )

### Creating objects from default values

All e-footprint classes also implement default values and a *from_defaults* method that allows for using a set a pre-defined default attributes and specifying the ones we want to specify through keyword arguments.

In [ ]:
Storage.default_values

In [ ]:
# Creating a storage object from defaults while specifying storage capacity using keyword arguments
print(Storage.from_defaults("2 TB SSD storage", storage_capacity=SourceValue(2 * u.TB)))

We can see from the above print that e-footprint objects have calculated attributes that are setup as empty and then computed by e-footprint when associated with a usage. More information on e-footprint objects’ calculated_attributes can be found in the [object reference section of the e-footprint documentation](https://boavizta.github.io/e-footprint/object_reference/).

### Creating objects from archetypes

Some e-footprint objects (Storage, Network and Hardware) also have archetypes that have their own set of default values:

In [ ]:
Storage.archetypes()

In [ ]:
print(Storage.hdd())

Apart from environmental and technical attributes, e-footprint objects can link to other e-footprint objects. For example, server objects have a storage attribute:

In [ ]:
server = Server.from_defaults(
    "server",
    server_type=ServerTypes.autoscaling(),
    power_usage_effectiveness=SourceValue(1.2 * u.dimensionless, Sources.HYPOTHESIS),
    average_carbon_intensity=SourceValue(100 * u.g / u.kWh, Sources.HYPOTHESIS),
    utilization_rate=SourceValue(0.9 * u.dimensionless, Sources.HYPOTHESIS),
    base_ram_consumption=SourceValue(300 * u.MB, Sources.HYPOTHESIS),
    base_compute_consumption=SourceValue(2 * u.cpu_core, Sources.HYPOTHESIS),
    storage=storage
)

print(server)

### Creating objects from builders connected to external data sources

Of course only relying on a single set of default values for creating our servers won’t get us far. That’s why e-footprint provides a builder class that connects to [Boavizta’s API](https://github.com/Boavizta/boaviztapi) to allow for the creation of servers from a cloud provider and an instance type.

In [ ]:
from efootprint.builders.hardware.boavizta_cloud_server import BoaviztaCloudServer

# Some attributes can only take specific values
for attribute, attribute_list_value in BoaviztaCloudServer.list_values.items():
    print(f"Possible values for {attribute}: {attribute_list_value}")

In [ ]:
# Moreover, some attributes depend on another attribute for their values
for attribute, attribute_conditional_dict in BoaviztaCloudServer.conditional_list_values.items():
    condition_attribute = attribute_conditional_dict['depends_on']
    print(f"Possible values for {attribute} depend on {condition_attribute}:\n")
    for condition_value, possible_values in attribute_conditional_dict["conditional_list_values"].items():
        if len(possible_values) > 10:
            values_to_print = possible_values[:5] + ["etc."]
        else:
            values_to_print = possible_values
        print(f"    Possible values when {condition_attribute} is {condition_value}: {values_to_print}")
    print("\n")

In [ ]:
# BoaviztaCloudServer still has quite a lot of default values but ones that are much easier to make hypothesis on, 
# like lifespan, server utilisation rate or power usage effectiveness
BoaviztaCloudServer.default_values

In [ ]:
# The most difficult environmental and technical attributes are retrieved from a call to BoaviztAPI:
print(BoaviztaCloudServer.from_defaults("Default Boavizta cloud server", storage=Storage.ssd(storage_capacity=SourceValue(32 * u.GB))))

## [Optional] Install services on your server

Manually creating job objects can get tricky because you have to specify how much RAM and compute the job uses on the server it runs on during its duration. That’s why e-footprint allows for the installation of services on servers, that will give access to higher-level job classes that compute these very technical attributes from simpler ones. For example, let’s install a video streaming service on our server:

### Video streaming service

In [ ]:
from efootprint.builders.services.video_streaming import VideoStreaming

VideoStreaming.default_values

In [ ]:
video_streaming_service = VideoStreaming.from_defaults("Video streaming service", server=server)

In [ ]:
# All services have a list of compatible job types, let’s check out the ones for video streaming:
VideoStreaming.compatible_jobs()

In [ ]:
# There’s only one so let’s use it !
from efootprint.builders.services.video_streaming import VideoStreamingJob

VideoStreamingJob.default_values

In [ ]:
print(VideoStreamingJob.list_values)

In [ ]:
# Now it’s easy to add a 1 hour 1080p streaming job to our streaming service
streaming_job = VideoStreamingJob.from_defaults(
    "streaming job", service=video_streaming_service, resolution=SourceObject("1080p (1920 x 1080)"), 
    video_duration=SourceValue(20 * u.min))

# For optimization purposes calculations are only made when usage data has been entered but we can force
# some of them to see what the VideoStreamingJob does.
streaming_job.update_dynamic_bitrate()
streaming_job.update_data_transferred()
streaming_job.update_compute_needed()
streaming_job.update_ram_needed()

print(streaming_job)

### Web application service

In the same vein, we can install a web application service on our server. e-footprint’s WebApplication service relies on the analysis of [Boavizta’s ecobenchmark project](https://github.com/Boavizta/ecobenchmark-applicationweb-backend).

In [ ]:
from efootprint.builders.services.web_application import WebApplication, WebApplicationJob

web_app_service = WebApplication("Web app", server=server, technology=SourceObject("php-symfony"))
web_app_job = WebApplicationJob.from_defaults("fetching web view", service=web_app_service)
web_app_job.update_compute_needed()
web_app_job.update_ram_needed()

print(web_app_service)
print(web_app_job)

## Define the user journey

This is the modeling of the average daily usage of the streaming platform in France:

In [ ]:
streaming_step = UsageJourneyStep(
    "20 min streaming",
    user_time_spent=SourceValue(20 * u.min, Sources.USER_DATA),
    jobs=[web_app_job, streaming_job])

video_upload_job = Job(
    "upload job", server=server, data_transferred=SourceValue(20 * u.MB, Sources.USER_DATA),
    data_stored=SourceValue(20 * u.MB, Sources.USER_DATA),
    request_duration=SourceValue(2 * u.s, Sources.HYPOTHESIS),
    compute_needed=SourceValue(1 * u.cpu_core, Sources.HYPOTHESIS),
    ram_needed=SourceValue(50 * u.MB, Sources.HYPOTHESIS))

upload_step = UsageJourneyStep(
    "1 min video capture then upload",
    user_time_spent=SourceValue(70 * u.s, Sources.USER_DATA),
    jobs=[web_app_job, video_upload_job])

The user journey is then simply a list of user journey steps:

In [ ]:
user_journey = UsageJourney("Mean video consumption user journey", uj_steps=[streaming_step, upload_step])

## Describe usage

An e-footprint usage pattern links a user journey to devices that run it, a network, a country, and the number of times the user journey gets executed hour by hour. 

In [ ]:
# Let’s build synthetic usage data by summing a linear growth with a sinusoidal fluctuation components, then adding daily variation
from datetime import datetime, timedelta

from efootprint.builders.time_builders import linear_growth_hourly_values

start_date = datetime.strptime("2025-01-01", "%Y-%m-%d")
timespan = 3 * u.year

linear_growth = linear_growth_hourly_values(timespan, start_value=5000, end_value=100000, start_date=start_date)
linear_growth.set_label("Hourly user journeys linear growth component")

linear_growth.plot()

In [ ]:
from efootprint.builders.time_builders import sinusoidal_fluct_hourly_values

sinusoidal_fluct = sinusoidal_fluct_hourly_values(
    timespan, sin_fluct_amplitude=3000, sin_fluct_period_in_hours=3 * 30 * 24, start_date=start_date)

lin_growth_plus_sin_fluct = (linear_growth + sinusoidal_fluct).set_label("Hourly user journeys linear growth with sinusoidal fluctuations")

lin_growth_plus_sin_fluct.plot()

In [ ]:
# Let’s add daily variations because people use the system less at night
from efootprint.builders.time_builders import daily_fluct_hourly_values

daily_fluct = daily_fluct_hourly_values(timespan, fluct_scale=0.8, hour_of_day_for_min_value=4, start_date=start_date)
daily_fluct.set_label("Daily volume fluctuation")

daily_fluct.plot(xlims=[start_date, start_date+timedelta(days=1)])

In [ ]:
hourly_user_journey_starts = lin_growth_plus_sin_fluct * daily_fluct
hourly_user_journey_starts.set_label("Hourly number of user journey started")

hourly_user_journey_starts.plot(xlims=[start_date, start_date + timedelta(days=7)])

In [ ]:
# Over 3 years the daily fluctuations color the area between daily min and max number of hourly user journeys
hourly_user_journey_starts.plot()

In [ ]:
network = Network(
        "WIFI network",
        bandwidth_energy_intensity=SourceValue(0.05 * u("kWh/GB"), Sources.TRAFICOM_STUDY))

usage_pattern = UsagePattern(
    "Daily video streaming consumption",
    usage_journey=user_journey,
    devices=[Device.laptop()],
    network=network,
    country=Countries.FRANCE(),
    hourly_usage_journey_starts=hourly_user_journey_starts)

system = System("System", usage_patterns=[usage_pattern], edge_usage_patterns=[])

## Results

### Computed attributes

Now all calculated_attributes have been computed:

In [ ]:
print(server)

### System footprint overview

In [ ]:
system.plot_footprints_by_category_and_object("System footprints.html")

### Object relationships graph

Hover over a node to get the numerical values of its environmental and technical attributes. For simplifying the graph the Network and Hardware nodes are not shown.

In [ ]:
from efootprint.utils.object_relationships_graphs import USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE, INFRA_VIEW_CLASSES_TO_IGNORE

usage_pattern.object_relationship_graph_to_file("object_relationships_graph_up_view.html", width="800px", height="500px",
    classes_to_ignore=USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE, notebook=True)

In [ ]:
usage_pattern.object_relationship_graph_to_file("object_relationships_graph_infra_view.html", width="800px", height="500px",
    classes_to_ignore=INFRA_VIEW_CLASSES_TO_IGNORE, notebook=True)

In [ ]:
usage_pattern.object_relationship_graph_to_file("object_relationships_graph_all_objects.html", width="800px", height="500px",
    classes_to_ignore=[], notebook=True)

### Calculus graph

Any e-footprint calculation can generate its calculation graph for full auditability. Hover on a calculus node to display its formula and numeric value.

In [ ]:
usage_pattern.devices_fabrication_footprint.calculus_graph_to_file(
    "device_population_fab_footprint_calculus_graph.html", width="800px", height="500px", notebook=True)

### Plotting an object’s hourly and cumulated CO2 emissions

In [ ]:
server.energy_footprint.plot()

In [ ]:
server.energy_footprint.plot(cumsum=True)

In [ ]:
system.total_footprint.plot(cumsum=True)

## Analysing the impact of a change
### Numeric input change
Any input change automatically triggers the computation of calculations that depend on the input. For example, let’s say that the average data download consumption of the streaming step decreased because of a change in default video quality:

In [ ]:
streaming_job.resolution = SourceObject("720p (1280 x 720)", Sources.USER_DATA)

In [ ]:
system.plot_emission_diffs("bandwith reduction.png")

### System structure change
Now let’s make a more complex change, like adding a conversation with a generative AI chatbot before streaming the video.
We will use e-footprint’s GenAIModel object that fetches LLM data like number of parameters from [EcoLogits](https://github.com/genai-impact/ecologits).

In [ ]:
# GenAI models need a GPU server
from efootprint.core.hardware.gpu_server import GPUServer
from efootprint.builders.services.generative_ai_ecologits import GenAIModel

llm_server = GPUServer.from_defaults("Inference GPU server", server_type=ServerTypes.serverless(), compute=SourceValue(16 * u.gpu),
                                    storage=Storage.ssd(storage_capacity=SourceValue(1 * u.TB)))
genai_model = GenAIModel.from_defaults(
    "Openai’s gpt-4o", server=llm_server, 
    provider=SourceObject("openai"), model_name=SourceObject("gpt-4o"))

In [ ]:
from efootprint.builders.services.generative_ai_ecologits import GenAIJob

genai_job = GenAIJob("LLM API call", genai_model, output_token_count= SourceValue(1000 * u.dimensionless))
llm_chat_step = UsageJourneyStep(
    "Chat with LLM to select video", user_time_spent=SourceValue(1 * u.min, Sources.HYPOTHESIS),
    jobs=[genai_job])

In [ ]:
# Adding the new step is simply an attribute update.
user_journey.uj_steps.append(llm_chat_step)

In [ ]:
system.plot_emission_diffs("LLM chat addition.png")

We can see that server energy footprint has been multiplied by more than 1000 and the rest of the impact is quite negligible. Good to know to make informed decisions ! Of course the impact is very much dependent on assumptions. If the LLM server ran on low-carbon electricity for example:

In [ ]:
llm_server.average_carbon_intensity = SourceValue(50 * u.g / u.kWh, Sources.HYPOTHESIS)
system.plot_emission_diffs("lower LLM inference carbon intensity.png")

## Recap of all System changes

In [ ]:
system.plot_emission_diffs("All system diffs.png", from_start=True)

## Making simulations of changes in the future

We’ve seen that you can make changes in your modeling and analyse the differences, but most likely the changes you’re contemplating will happen at some point in the future. Let’s model a change in the future thanks to e-footprint’s ModelingUpdate object !

In [ ]:
# Let’s first revert the system to its state before changes
# We can make optimized batch changes by using the ModelingUpdate object, that is also used to make simulations of changes in the future
from efootprint.abstract_modeling_classes.modeling_update import ModelingUpdate

ModelingUpdate([
    [user_journey.uj_steps, [streaming_step, upload_step]],
    [llm_server.average_carbon_intensity, SourceValue(300 * u.g / u.kWh, Sources.HYPOTHESIS)],
    [streaming_job.resolution, SourceObject("1080p (1920 x 1080)")]
])

system.plot_footprints_by_category_and_object("System footprints after reset.html")

In [ ]:
# To create a simulation, which is a change in the future, simply set ModelingUpdate’s simulation_date parameter
import pytz
from datetime import timezone

simulation = ModelingUpdate([[user_journey.uj_steps, [streaming_step, upload_step, llm_chat_step]]],
                           simulation_date=pytz.utc.localize(start_date) + timedelta(days=365))

In [ ]:
system.total_footprint.plot(cumsum=True)

In [ ]:
llm_server.energy_footprint.plot(cumsum=True)

In [ ]:
# The system state is reset to baseline after the simulation.
# For example, our LLM server has no energy footprint since it is not used in the baseline modeling
llm_server.energy_footprint

In [ ]:
# To set simulation values, use ModelingUpdate’s set_updated_values method
simulation.set_updated_values()
llm_server.energy_footprint

In [ ]:
# Conversely, pre-update values are reset using ModelingUpdate’s reset_values method
simulation.reset_values()
llm_server.energy_footprint

# [Optional] Model edge devices
The above modeling reflects a centralized web logic: the input usage is a business input (typically, the number of visits on a website), and the infrastructure has to be sized to meet the demand. But there are cases where the usage is actually proportional to a number of devices, typically when trying to model the impact of selling digital devices, like servers, smartphones or video game consoles. In this case, e-footprint edge objects allow for the description of usage per device, and number of devices sold / produced over the modeling period.

## Define the edge device
The EdgeStorage and EdgeDevice objects are similar to the Storage and Server objects, but simpler because they don’t have to include any autoscaling logic.

In [ ]:
from efootprint.core.hardware.edge_storage import EdgeStorage
from efootprint.core.hardware.edge_device import EdgeDevice

edge_storage = EdgeStorage(
    "Edge SSD storage",
    carbon_footprint_fabrication_per_storage_capacity=SourceValue(160 * u.kg / u.TB),
    power_per_storage_capacity=SourceValue(1.3 * u.W / u.TB),
    lifespan=SourceValue(6 * u.years),
    idle_power=SourceValue(0.1 * u.W),
    storage_capacity=SourceValue(256 * u.GB),
    base_storage_need=SourceValue(10 * u.GB),
)

edge_device = EdgeDevice(
    "Edge device",
    carbon_footprint_fabrication=SourceValue(60 * u.kg),
    power=SourceValue(30 * u.W),
    lifespan=SourceValue(8 * u.year),
    idle_power=SourceValue(5 * u.W),
    ram=SourceValue(16 * u.GB),
    compute=SourceValue(8 * u.cpu_core),
    power_usage_effectiveness=SourceValue(1.0 * u.dimensionless),
    utilization_rate=SourceValue(0.8 * u.dimensionless),
    base_ram_consumption=SourceValue(1 * u.GB),
    base_compute_consumption=SourceValue(0.1 * u.cpu_core),
    storage=edge_storage
)

## Define the edge usage journey with the processes that run on the edge device
The EdgeUsageJourney object associate a list of RecurrentEdgeProcesses to an EdgeDevice object, over a usage span. The RecurrentEdgeProcess object describes the usage of compute, RAM and storage on the EdgeDevice object by describing hourly usage in a canonical week starting at midnight on a Monday local time.

In [ ]:
import numpy as np
from pint import Quantity

from efootprint.abstract_modeling_classes.source_objects import SourceRecurringValues
from efootprint.core.usage.recurrent_edge_process import RecurrentEdgeProcess
from efootprint.core.usage.edge_usage_journey import EdgeUsageJourney

edge_process = RecurrentEdgeProcess(
    "Edge process",
    recurrent_compute_needed=SourceRecurringValues(
        Quantity(np.array([1] * 168, dtype=np.float32), u.cpu_core)), # 7 * 24 = 168 hours in the canonical week
    recurrent_ram_needed=SourceRecurringValues(
        Quantity(np.array([2] * 168, dtype=np.float32), u.GB)),
    recurrent_storage_needed=SourceRecurringValues(
        Quantity(np.array([200] * 168, dtype=np.float32), u.kB))
)

edge_usage_journey = EdgeUsageJourney(
    "Edge usage journey",
    edge_processes=[edge_process],
    edge_device=edge_device,
    usage_span=SourceValue(6 * u.year)
)

## Define the edge usage pattern
The EdgeUsagePattern object specifies the number of edge devices that are emitted in a given countrey across time, through its *hourly_edge_usage_journey_starts* parameter. It could be for example the sales projection of a video game console seller. It is then linked to a System through System’s edge_usage_patterns parameter, just like web usage patterns are linked to a System through the usage_patterns parameter. That way, web and edge objects can coexist in the same simulation.

In [ ]:
from efootprint.builders.time_builders import create_hourly_usage_from_frequency
from efootprint.core.usage.edge_usage_pattern import EdgeUsagePattern

edge_usage_pattern = EdgeUsagePattern(
    "Edge usage pattern",
    edge_usage_journey=edge_usage_journey,
    country=Countries.FRANCE(),
    hourly_edge_usage_journey_starts=create_hourly_usage_from_frequency(
        timespan=5 * u.year, input_volume=10, frequency='weekly',
        active_days=[0, 1, 2, 3, 4, 5], hours=[9, 10, 11, 12, 15, 16, 17, 18, 19])
)

system.edge_usage_patterns = [edge_usage_pattern]

system.plot_footprints_by_category_and_object("System with edge objects footprints.html")

In [ ]:
system.plot_emission_diffs("Add edge objects.png")

In [ ]:
system.object_relationship_graph_to_file("system_with_edge_objects_graph_up_view.html", width="800px", height="500px",
    classes_to_ignore=USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE, notebook=True)

In [ ]:
system.object_relationship_graph_to_file("system_with_edge_objects_graph_infra_view.html", width="800px", height="500px",
    classes_to_ignore=INFRA_VIEW_CLASSES_TO_IGNORE, notebook=True)